与使用 Query、Path 和 Body 在路径操作函数中声明额外的校验和元数据的方式相同，你可以使用 Pydantic 的 Field 在 Pydantic 模型内部声明校验和元数据。

首先，你必须导入Field，然后，你可以对模型属性使用 Field：

注意，Field 是直接从 pydantic 导入的，而不是像其他的（Query，Path，Body 等）都从 fastapi 导入。

Field 的工作方式和 Query、Path 和 Body 相同，包括它们的参数等等也完全相同。

你可以在 Field、Query、Path、Body 中声明额外的信息。这些信息将包含在生成的 JSON Schema 中。

In [2]:
import uvicorn
from typing import Annotated
from fastapi import Body, FastAPI
from pydantic import BaseModel, Field

app = FastAPI()


class Item(BaseModel):
    name: str
    description: str | None = Field(
        default=None, title="The description of the item", max_length=300
    )
    price: float = Field(gt=0, description="The price must be greater than zero")
    tax: float | None = None


@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Annotated[Item, Body(embed=True)]):
    results = {"item_id": item_id, "item": item}
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [15404]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:56186 - "PUT /items/5 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15404]


## requests库

In [ ]:
url = 'http://127.0.0.1:8009/items/5' 
data = {
    "item":{"name": "细胞生物学", "description": "考研书籍", "price":35.8, "tax":0.6}
}
res = requests.put(url, json=data) 
res.text

# 在另一个ipynb文件中运行代码，会得到
# '{"item_id":5,"item":{"name":"细胞生物学","description":"考研书籍","price":35.8,"tax":0.6}}'

## 模式的额外信息 - 例子

您可以在JSON模式中定义额外的信息。

一个常见的用例是添加一个将在文档中显示的example。

有几种方法可以声明额外的 JSON 模

您可以使用 Config 和 schema_extra 为Pydantic模型声明一个示例

这些额外的信息将按原样添加到输出的JSON模式中。式信息。

In [3]:
import uvicorn
from typing import Annotated
from fastapi import Body, FastAPI
from pydantic import BaseModel, Field

app = FastAPI()


class Item(BaseModel):
    name: str
    description: str | None = Field(
        default=None, title="The description of the item", max_length=300
    )
    price: float = Field(gt=0, description="The price must be greater than zero")
    tax: float | None = None
    model_config = {
        "json_schema_extra": {
            "examples": [
                {
                    "name": "Foo",
                    "description": "A very nice Item",
                    "price": 35.4,
                    "tax": 3.2,
                }
            ]
        }
    }


@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Annotated[Item, Body(embed=True)]):
    results = {"item_id": item_id, "item": item}
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [15404]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:56283 - "PUT /items/6 HTTP/1.1" 200 OK
INFO:     127.0.0.1:56285 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56285 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:56360 - "PUT /items/7 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15404]


## fastapi提供的docs界面

在根目录输入/docs 即可打开。

打开后选中 PUT /items/ 这个接口，打开下拉窗口

可以看到 Parameters下面 item_id 被标红星，Required

在Request body Required下方的大输入框中可以看到例子已经在里面了
{
  "item": {
    "description": "A very nice Item",
    "name": "Foo",
    "price": 35.4,
    "tax": 3.2
  }
}

## Field 的附加参数

在 Field, Path, Query, Body 和其他你之后将会看到的工厂函数，你可以为JSON 模式声明额外信息，你也可以通过给工厂函数传递其他的任意参数来给JSON 模式声明额外信息，比如增加 example:

请记住，传递的那些example不会添加任何验证，只会添加注释，用于文档的目的。

In [4]:
import uvicorn
from typing import Annotated
from fastapi import Body, FastAPI
from pydantic import BaseModel, Field

app = FastAPI()

class Item(BaseModel):
    name: str = Field(examples=["Foo"])
    description: str | None = Field(default=None, examples=["A very nice Item"])
    price: float = Field(examples=[35.4])
    tax: float | None = Field(default=None, examples=[3.2])


@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Annotated[Item, Body(embed=True)]):
    results = {"item_id": item_id, "item": item}
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [15404]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:56410 - "PUT /items/6 HTTP/1.1" 200 OK
INFO:     127.0.0.1:56414 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56414 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:56413 - "PUT /items/5 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15404]


In [ ]:
使用Field声明示例与使用 Config 和 schema_extra 为Pydantic模型声明示例效果相同。

## Body 额外参数

你可以通过传递额外信息给 Field 同样的方式操作Path, Query, Body等。

比如，你可以将请求体的一个 example 传递给 Body:

In [10]:
import uvicorn
from typing import Annotated
from fastapi import Body, FastAPI
from pydantic import BaseModel, Field

app = FastAPI()

class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None

body_examples = {
    "name": "细胞生物学",
    "description": "考研书籍",
    "price": 35.8,
    "tax": 0.6,
}


@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Annotated[Item, Body(embed=True,example=body_examples)]):
    results = {"item_id": item_id, "item": item}
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [15404]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:56677 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56677 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15404]


关于 example 和 examples...
如果参数用example，比如Body(embed=True,example=body_examples)
那么就只需要传入一个字典，这个字典就是一个例子。

如果参数用examples，比如Body(embed=True,examples=body_examples)
那么例子必须被放入列表中，可以放多个，但是docs中只会显示第一个。

## 额外数据类型

到目前为止，您一直在使用常见的数据类型，如:

int
float
str

但是您也可以使用更复杂的数据类型。

下面是一些你可以使用的其他数据类型:

UUID:
一种标准的 "通用唯一标识符" ，在许多数据库和系统中用作ID。
在请求和响应中将以 str 表示。
datetime.datetime:
一个 Python datetime.datetime.
在请求和响应中将表示为 ISO 8601 格式的 str ，比如: 2008-09-15T15:53:00+05:00.
datetime.date:
Python datetime.date.
在请求和响应中将表示为 ISO 8601 格式的 str ，比如: 2008-09-15.
datetime.time:
一个 Python datetime.time.
在请求和响应中将表示为 ISO 8601 格式的 str ，比如: 14:23:55.003.
datetime.timedelta:
一个 Python datetime.timedelta.
在请求和响应中将表示为 float 代表总秒数。
Pydantic 也允许将其表示为 "ISO 8601 时间差异编码"
bool

In [13]:
import uvicorn
from datetime import datetime, time, timedelta
from typing import Annotated
from uuid import UUID
from fastapi import Body, FastAPI

app = FastAPI()

@app.put("/items/{item_id}")
async def read_items(
    item_id: UUID,
    start_datetime: Annotated[datetime | None, Body()] = None,
    end_datetime: Annotated[datetime | None, Body()] = None,
    repeat_at: Annotated[time | None, Body()] = None,
    process_after: Annotated[timedelta | None, Body()] = None,
):
    start_process = start_datetime + process_after
    duration = end_datetime - start_process
    return {
        "item_id": item_id,
        "start_datetime": start_datetime,
        "end_datetime": end_datetime,
        "repeat_at": repeat_at,
        "process_after": process_after,
        "start_process": start_process,
        "duration": duration,
    }

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [15404]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:49157 - "PUT /items/bb745154-83b3-4a4f-8f24-b8546ee1a4b3 HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:49166 - "PUT /items/bb745154-83b3-4a4f-8f24-b8546ee1a4b3 HTTP/1.1" 200 OK
INFO:     127.0.0.1:49175 - "PUT /items/bb745154-83b3-4a4f-8f24-b8546ee1a4b3 HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15404]


## requests库

In [12]:
import uuid
 
# 生成一个随机的 UUID
random_uuid = uuid.uuid4()
print(random_uuid)

b6abc40a-2d46-452f-aff8-fdcf22e3dfa7


In [ ]:
url = 'http://127.0.0.1:8009/items/b6abc40a-2d46-452f-aff8-fdcf22e3dfa7' 
data = {
  "start_datetime": "2024-01-24 07:42:54",
  "end_datetime": "2024-01-24 07:43:54",
  "repeat_at": "07:41:54",
  "process_after": 23.567
}
res = requests.put(url, json=data) 
res.text

# 在另一个ipynb文件中运行代码，会得到
# {
#   "item_id": "b6abc40a-2d46-452f-aff8-fdcf22e3dfa7",
#   "start_datetime": "2024-01-24T07:41:54",
#   "end_datetime": "2024-01-24T07:42:54",
#   "repeat_at": "07:41:54",
#   "process_after": 23.567,
#   "start_process": "2024-01-24T07:42:17.567000",
#   "duration": 36.433
# }

fastapi 会帮我们检查是否是符合UUID、datetime、date、time 的格式

比如  "2024-01-24 07:41:54" 是可以的，"2024-01-32 07:41:54" 是不行的，因为1月没有第32天。

"07:41" 会被当做 "07:41:00"